# École Polytechnique de Montréal
Département Génie Informatique et Génie Logiciel
INF8460 – Traitement automatique de la langue naturelle

### Prof. Amal Zouaq
### Chargé de laboratoire: Félix Martel


# INF8460 - TP2

## Objectifs

•	Explorer les modèles d’espaces vectoriels comme représentations distribuées de la sémantique des mots et des documents

•	Comprendre différentes mesures de distance entre vecteurs de documents et de mots

•	Utiliser un modèle de langue n-gramme de caractères et l’algorithme Naive Bayes pour l’analyse de sentiments dans des revues de films (positives, négatives)


## 1. Prétraitement

Le jeu de données est séparé en deux répertoires `train/`et `test`, chacun contenant eux-mêmes deux sous-répertoires `pos/` et `neg/` pour les revues positives et négatives. Un fichier `readme` décrit plus précisément les données.

Commencez par lire ces données, en gardant séparées les données d'entraînement et de test.

In [1]:
import copy
import glob
import math
import nltk
import numpy as np
import re
import string
import time

from scipy.spatial import distance
from collections import defaultdict
from nltk.corpus import stopwords

Lire les données 

L'odre des sections de data: [test_data_pos, test_data_neg, train_data_pos, train_data_neg, train_data_unsup]

In [2]:
def read_dataset():
    path = "./data/aclImdb"
    test_endpath = "/test"
    train_endpath = "/train"

    files_test_pos = glob.glob(path + test_endpath + "/pos/*.txt")
    files_test_neg= glob.glob(path + test_endpath + "/neg/*.txt")
    files_train_pos = glob.glob(path + train_endpath + "/pos/*.txt")
    files_train_neg = glob.glob(path + train_endpath + "/neg/*.txt")
    files_train_unsup = glob.glob(path + train_endpath + "/unsup/*.txt")
    sections = [files_test_pos, files_test_neg, files_train_pos, files_train_neg, files_train_unsup]

    #Each data is a list of strings(reviews)
    data = [[], [], [], [], []]
    for i, section in enumerate(sections):
        for file in section:
            try:
                with open(file, encoding="utf8") as f:
                    data[i].append(f.read()) 

            except IOError as exc:
                if exc.errno != errno.EISDIR:
                    raise
    
    return data

In [3]:
start = time.time()
data = read_dataset()
end = time.time()

print("Données lues")
print("Temps d'exécution: " + str(end-start))

Données lues
Temps d'exécution: 104.65009880065918


**a)** Créez la fonction `clean_doc()` qui effectue les pré-traitements suivants : segmentation en mots ; 
suppression des signes de ponctuations ; suppression des mots qui contiennent des caractères autres qu’alphabétiques ; 
suppression des mots qui sont connus comme des stop words ; suppression des mots qui ont une longueur de 1 caractère. Ensuite, appliquez-la à vos données.

Les stop words peuvent être obtenus avec `from nltk.corpus import stopwords`. Vous pourrez utiliser des [expressions régulières](https://docs.python.org/3.7/howto/regex.html).

In [4]:
stop_words = stopwords.words('english')

def preprocess_review(review):
    review_no_punct = review.translate(str.maketrans('', '', string.punctuation))
    words = nltk.word_tokenize(review_no_punct)

    pre_words = []
    for word in words:
        word = word.lower()
        if word.isalpha() and (word not in stop_words) and len(word) > 1:
            pre_words.append(word)

    return pre_words

In [5]:
def clean_doc(data):
    pre_data = [[],[],[],[],[]]
    for i, dataset in enumerate(data):
        for review in dataset:
            pre_data[i].append(preprocess_review(review))
    
    return pre_data

In [6]:
start = time.time()
preprocessed_data = clean_doc(data)
end = time.time()

print("Données pré-traitées")
print("Temps d'exécution: " + str(end-start))

Données pré-traitées
Temps d'exécution: 94.96665501594543


**b)**	Créez la fonction `build_voc()` qui extrait les unigrammes de l’ensemble d’entraînement et conserve ceux qui ont une fréquence d’occurrence de 5 au moins et imprime le nombre de mots dans le vocabulaire. Sauvegardez-le dans un fichier `vocab.txt` (un mot par ligne).

In [7]:
def build_voc():  
    preprocessed_data_train = preprocessed_data[2] + preprocessed_data[3] + preprocessed_data[4]
    
    counts = defaultdict(int)
    for review in preprocessed_data_train:
        for unigram in review:
            counts[unigram] += 1

    unigram = defaultdict(int)
    with open("vocab.txt", "w+", encoding="utf8") as f:
        for count in counts.items():
            if (count[1] >= 5):
                f.write(count[0] + '\n')
                unigram[count[0]] = count[1]

    print("Nombre de mots: " + str(len(unigram)))
    return unigram

In [8]:
unigram_train = build_voc()

Nombre de mots: 55234


**c)** Vous devez créer une fonction `get_top_unigrams(n)` qui retourne les $n$ unigrammes les plus fréquents et les affiche, puis l'appeler avec $n=10$.

In [9]:
def get_top_unigrams(n):
    sorted_unigram_train = sorted(unigram_train.items(), key=lambda v: v[1], reverse=True)
    top_unigrams_train_word = [word[0] for word in sorted_unigram_train]
    
    return top_unigrams_train_word[:n]

In [10]:
top_unigrams_train = get_top_unigrams(10)

print("Top 10 unigrammes les plus fréquents")
print(top_unigrams_train)

Top 10 unigrammes les plus fréquents
['br', 'movie', 'film', 'one', 'like', 'good', 'even', 'would', 'time', 'really']


**d)**	Vous devez créer une fonction `get_top_unigrams_per_cls(n, cls)` qui retourne les $n$ unigrammes les plus fréquents de la classe `cls` (pos ou neg) et les affiche.

In [11]:
def convert_cls_data_defaultdic(preprocessed_data_per_cls):
    counts = defaultdict(int)
    for review in preprocessed_data_per_cls:
        for unigram in review:
            counts[unigram] += 1
    
    return counts

In [12]:
def get_top_unigrams_per_cls(n, cls):
    if (cls == 'pos'):
        preprocessed_data_per_cls = preprocessed_data[2]
    elif (cls == 'neg'):
        preprocessed_data_per_cls = preprocessed_data[3]
    unigram_per_cls = convert_cls_data_defaultdic(preprocessed_data_per_cls)
    sorted_unigram_per_cls = sorted(unigram_per_cls.items(), key=lambda v: v[1], reverse=True)
    
    top_unigrams_per_cls_word = [word[0] for word in sorted_unigram_per_cls]
    return top_unigrams_per_cls_word[:n]

**e)**	Affichez les 10 unigrammes les plus fréquents dans la classe positive :

In [13]:
top_10_unigram_pos = get_top_unigrams_per_cls(10, 'pos')

print("Top 10 unigrammes les plus fréquents dans la classe positive")
print(top_10_unigram_pos)

Top 10 unigrammes les plus fréquents dans la classe positive
['br', 'film', 'movie', 'one', 'like', 'good', 'story', 'great', 'time', 'see']


**f)**	Affichez les 10 unigrammes les plus fréquents dans la classe négative :

In [14]:
top_10_unigram_neg = get_top_unigrams_per_cls(10, 'neg')

print("Top 10 unigrammes les plus fréquents dans la classe négative")
print(top_10_unigram_neg)

Top 10 unigrammes les plus fréquents dans la classe négative
['br', 'movie', 'film', 'one', 'like', 'even', 'good', 'bad', 'would', 'really']


## 2. Matrices de co-occurence

Pour les matrices de cette section, vous pourrez utiliser [des array `numpy`](https://docs.scipy.org/doc/numpy/reference/arrays.ndarray.html) ou des DataFrame [`pandas`](https://pandas.pydata.org/pandas-docs/stable/). 

Ressources utiles :  le [*quickstart tutorial*](https://numpy.org/devdocs/user/quickstart.html) de numpy et le guide [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html).

### 2.1 Matrice document × mot et TF-IDF


Soit $X \in \mathbb{R}^{m \times n}$ une matrice de $m$ documents et $n$ mots, telle que $X_{i,j}$ contient la fréquence d'occurrence du terme $j$ dans le document $i$ :

$$\textbf{rowsum}(X, d) = \sum_{j=1}^{n}X_{dj}$$

$$\textbf{TF}(X, d, t) = \frac{X_{d,t}}{\textbf{rowsum}(X, d)}$$

$$\textbf{IDF}(X, t) = \log\left(\frac{m}{|\{d : X_{d,t} > 0\}|}\right)$$

$$\textbf{TF-IDF}(X, d, t) = \textbf{TF}(X, d, t) \cdot \textbf{IDF}(X, t)$$


En utilisant le même vocabulaire de 5 000 unigrammes, vous devez représenter les documents dans une matrice de co-occurrence document × mot $M(d, w)$  et les pondérer avec la mesure TF-IDF.

La première étape est d'avoir le vocabulaire de 5000 unigrammes. 

In [15]:
def get_top_unigram_pos_neg(n):
    preprocessed_data_train_pos_neg = preprocessed_data[2] + preprocessed_data[3]
    unigram_train_pos_neg = convert_cls_data_defaultdic(preprocessed_data_train_pos_neg)
    sorted_unigram_train_pos_neg = sorted(unigram_train_pos_neg.items(), key=lambda v: v[1], reverse=True)
    
    return sorted_unigram_train_pos_neg[:n]

In [16]:
unigrams_top_5000 = get_top_unigram_pos_neg(5000)
unigrams_top_5000_words = list(dict(unigrams_top_5000).keys())

In [17]:
data_train_pos_neg = np.asarray(data[2] + data[3])

Maintenant, nous allons construire la matrice bag of words.

In [18]:
def count_BOW(top_unigrams, data_train):
    matrix_BOW = np.zeros((data_train.size, len(top_unigrams)))    

    for i, review in enumerate(data_train):
        pre_review = np.array(preprocess_review(review))
        unique, counts = np.unique(pre_review, return_counts = True)
        for k, word in enumerate(unique):
            if word in top_unigrams:
                j = top_unigrams.index(word)
                matrix_BOW[i][j] += counts[k]

    return matrix_BOW

In [19]:
start = time.time()
matrix_BOW = count_BOW(unigrams_top_5000_words, data_train_pos_neg)
end = time.time()

print("Matrice bag of words est finie")
print("Temps d'exécution: " + str(end - start))

Matrice bag of words est finie
Temps d'exécution: 149.92942142486572


In [20]:
def create_matrix_TFIDF(matrix_count_bow, data_train):
    sum_cols = np.sum(matrix_count_bow, axis=0)
    idf = np.log(data_train.size/sum_cols)
    idf = np.absolute(idf)

    sum_rows = np.sum(matrix_count_bow, axis=1) 
    tf = matrix_count_bow / sum_rows.reshape((-1,1))
    matrix_TFIDF = tf * idf
    
    return matrix_TFIDF

In [21]:
start = time.time()
matrix_TFIDF = create_matrix_TFIDF(matrix_BOW, data_train_pos_neg)
end = time.time()

print("Matrice TF-IDF est finie")
print("Temps d'exécution: " + str(end - start))

Matrice TF-IDF est finie
Temps d'exécution: 1.5753278732299805


## 2.2 Matrice mot × mot et PPMI (*positive pointwise mutual information*)

Vous devez calculer la métrique PPMI. Pour une matrice $m \times n$ $X$ :


$$\textbf{colsum}(X, j) = \sum_{i=1}^{m}X_{ij}$$

$$\textbf{sum}(X) = \sum_{i=1}^{m}\sum_{j=1}^{n} X_{ij}$$

$$\textbf{expected}(X, i, j) = 
\frac{
  \textbf{rowsum}(X, i) \cdot \textbf{colsum}(X, j)
}{
  \textbf{sum}(X)
}$$


$$\textbf{pmi}(X, i, j) = \log\left(\frac{X_{ij}}{\textbf{expected}(X, i, j)}\right)$$

$$\textbf{ppmi}(X, i, j) = 
\begin{cases}
\textbf{pmi}(X, i, j) & \textrm{if } \textbf{pmi}(X, i, j) > 0 \\
0 & \textrm{otherwise}
\end{cases}$$


**a)**	A partir des textes du corpus d’entrainement (neg *et* pos), vous devez construire une matrice de co-occurrence mot × mot $M(w,w)$ qui contient les 5000 unigrammes les plus fréquents. 

In [22]:
def calculate_matrix_word_word():
    matrix_ww = np.zeros((5000, 5000))
    dict_unigrams = dict(unigrams_top_5000)
    preprocessed_data_pos_neg = np.array(preprocessed_data[2] + preprocessed_data[3])
    for review in preprocessed_data_pos_neg:
        for i, word in enumerate(review):
            if word in dict_unigrams:
                index = unigrams_top_5000_words.index(word)
                for neighbor in np.concatenate((review[max(0,i-5):i], review[i+1:min(i+6,len(review))])):
                    if neighbor in dict_unigrams:
                        index_neighbor = unigrams_top_5000_words.index(neighbor)
                        matrix_ww[index][index_neighbor] += 1
                        matrix_ww[index_neighbor][index] += 1

                    
    return matrix_ww

In [23]:
start = time.time()
matrix_ww = calculate_matrix_word_word()
end = time.time()

print("Matrice de co-occurence terminée")
print("Temps d'exécution: " + str(end - start))

Matrice de co-occurence terminée
Temps d'exécution: 387.26822328567505


In [24]:
print(matrix_ww)

[[3.4500e+04 1.7364e+04 1.5606e+04 ... 2.0000e+01 2.6000e+01 1.2000e+01]
 [1.7364e+04 1.3728e+04 3.7700e+03 ... 1.2000e+01 1.2000e+01 1.6000e+01]
 [1.5606e+04 3.7700e+03 9.7880e+03 ... 8.0000e+00 1.0000e+01 2.2000e+01]
 ...
 [2.0000e+01 1.2000e+01 8.0000e+00 ... 4.0000e+00 0.0000e+00 0.0000e+00]
 [2.6000e+01 1.2000e+01 1.0000e+01 ... 0.0000e+00 4.0000e+00 0.0000e+00]
 [1.2000e+01 1.6000e+01 2.2000e+01 ... 0.0000e+00 0.0000e+00 1.2000e+01]]


**b)**	Vous devez créer une fonction `calculate_PPMI` qui prend la matrice $M(w,w)$ et la transforme en une matrice $M’(w,w)$ avec les valeurs PPMI.

In [25]:
def calculate_PMI(matrix_ww):
    
    sum_cols = np.sum(matrix_ww, axis=0)
    sum_rows = np.sum(matrix_ww, axis=1) 
    sum_matrix = np.sum(matrix_ww)
    expected = sum_cols.reshape((-1,1)) @ sum_rows.reshape((1,-1)) / sum_matrix
    matrix_PMI = np.log2(matrix_ww/expected)
    
    return matrix_PMI

In [26]:
def calculate_PPMI():
    matrix_PMI = calculate_PMI(matrix_ww)
    matrix_PPMI = matrix_PMI.clip(0)
    
    return matrix_PPMI

In [27]:
start = time.time()
matrix_PPMI = calculate_PPMI()
end = time.time()

print("Matrice PPMI terminée")
print("Temps d'exécution: " + str(end - start))

Matrice PPMI terminée
Temps d'exécution: 1.4935357570648193


c:\users\mohamed ali elakhras\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log2
  import sys


In [28]:
print(matrix_PPMI)

[[0.82546523 0.20352149 0.2289324  ... 0.         0.09755701 0.        ]
 [0.20352149 0.23309692 0.         ... 0.         0.         0.        ]
 [0.2289324  0.         0.10388032 ... 0.         0.         0.29683612]
 ...
 [0.         0.         0.         ... 6.76420194 0.         0.        ]
 [0.09755701 0.         0.         ... 0.         7.0430781  0.        ]
 [0.         0.         0.29683612 ... 0.         0.         8.41268956]]


## 3. Mesures de similarité

En utilisant le module [scipy.spatial.distance](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html),  définissez des fonctions pour calculer les métriques suivantes :

**Distance Euclidienne**

La distance euclidienne entre deux vecteurs $u$ et $v$ de dimension $n$ est

$$\textbf{euclidean}(u, v) = 
\sqrt{\sum_{i=1}^{n}|u_{i} - v_{i}|^{2}}$$

En deux dimensions, cela correspond à la longueur de la ligne droite entre deux points.

**a)** Implémentez la fonction `get_euclidean_distance(v1 ,v2)` qui retourne la distance euclidienne entre les vecteurs v1 et v2.

In [29]:
def get_euclidean_distance(v1 ,v2):
    return distance.euclidean(v1, v2)

**Distance Cosinus**


La distance cosinus entre deux vecteurs $u$ et $v$ de dimension $n$ s'écrit :

$$\textbf{cosine}(u, v) = 
1 - \frac{\sum_{i=1}^{n} u_{i} \cdot v_{i}}{\|u\|_{2} \cdot \|v\|_{2}}$$

Le terme de droite dans la soustraction mesure l'angle entre $u$ et $v$; on l'appelle la *similarité cosinus* entre $u$ et $v$.

**b)** Implémentez la fonction `get_cosinus_distance(v1, v2)` qui retourne la distance cosinus entre les vecteurs v1 et v2.

In [30]:
def get_cosinus_distance(v1, v2):
    return distance.cosine(v1, v2)

**c)** Implémentez la fonction `get_most_similar_PPMI(word, metric, n)` qui prend un mot en entrée et une mesure de distance et qui retourne les n mots les plus similaires selon la mesure. Les mesures à tester sont : la distance euclidienne et la distance cosinus implantées ci-dessus. Le vecteur du mot word doit être extrait de la matrice $M’(w,w)$.

In [31]:
def get_most_similar_PPMI(word, metric, n):
    word_unigrams = list(dict(unigrams_top_5000).keys())
    index_word = word_unigrams.index(word)
    neighbors = matrix_PPMI[index_word]
    
    distance = {}
    for i, word_ww in enumerate(matrix_PPMI):
        if metric == "euclidean":
            distance[word_unigrams[i]] = get_euclidean_distance(neighbors, word_ww)
        elif metric == "cosine":
            distance[word_unigrams[i]] = get_cosinus_distance(neighbors, word_ww)
    
    sorted_distance = sorted(distance.items(), key=lambda v: v[1], reverse=False)
    sorted_distance_word = [word[0] for word in sorted_distance]
    
    return sorted_distance_word[1:n+1]

**d)** Trouvez les 5 mots les plus similaires au mot « bad » et affichez-les, pour chacune des deux distances. Commentez.

In [32]:
words_sim_bad = get_most_similar_PPMI("bad", "euclidean", 5)

print("5 mots les plus similaires au mot bad avec la distance euclidéenne")
print(words_sim_bad)

5 mots les plus similaires au mot bad avec la distance euclidéenne
['movie', 'good', 'really', 'even', 'like']


In [33]:
words_sim_bad = get_most_similar_PPMI("bad", "cosine", 5)

print("5 mots les plus similaires au mot bad avec la distance cosine")
print(words_sim_bad)

5 mots les plus similaires au mot bad avec la distance cosine
['acting', 'awful', 'terrible', 'horrible', 'effects']


*-> Commentez ici<-*

La métrique cosine donne de bons résultats. En effet, lorsqu'on regarde les 5 premiers mots, nous avons trois synonymes du mots bad (awful, terrible et horrible). Les deux autres mots ont rapport au mot bad. L'adjectif bad vient ajouter une connotation négative à acting et à effects. On peut conclure que les spectateurs n'ont pas apprécié le jeu d'acteurs et les effets spéciaux. Aussi, on peut voir la proximité des adjectifs awful, terrible et horrible. Cela montre à quel point les spectateurs ont une mauvaise expérience.

La métrique euclidéenne donne de moins bons résultats que la distance cosine. En effet, parmis les 5 mots les plus similaire, on retrouve deux antonymes au mot bad (good et like). Aussi, on retrouve le mot even qui a une connotation neutre. 

**e)** Implémentez la fonction `get_most_similar_TFIDF(word, metric, n)` qui prend un mot en entrée et une mesure de distance et qui retourne les n mots les plus similaires selon la mesure. Les mesures à tester sont : la distance euclidienne et la distance cosinus implantées ci-dessus. Le vecteur du mot word doit être extrait de la matrice $M(d,w)$.

In [34]:
def get_most_similar_TFIDF(word, metric, n):
    word_unigrams = list(dict(unigrams_top_5000).keys())
    index_word = word_unigrams.index(word)
    word_documents = matrix_TFIDF[:,index_word]
    
    distance = {}
    for i, word_TFIDF in enumerate(matrix_TFIDF.T):
        if metric == "euclidean":
            distance[word_unigrams[i]] = get_euclidean_distance(word_documents, word_TFIDF)
            sorted_distance = sorted(distance.items(), key=lambda v: v[1], reverse=True)
        elif metric == "cosine":
            distance[word_unigrams[i]] = get_cosinus_distance(word_documents, word_TFIDF)
            sorted_distance = sorted(distance.items(), key=lambda v: v[1], reverse=False)

    sorted_distance_word = [word[0] for word in sorted_distance]

    if metric == "euclidean":
        return sorted_distance_word[:n]
    elif metric == "cosine":
        return sorted_distance_word[1:n+1]

**f)** Trouvez les 5 mots les plus similaires au mot « bad » et affichez-les, pour chacune des deux distances. Commentez

In [35]:
words_sim_bad = get_most_similar_TFIDF("bad", "euclidean", 5)
print(words_sim_bad)

['br', 'episode', 'movie', 'show', 'series']


In [36]:
words_sim_bad = get_most_similar_TFIDF("bad", "cosine", 5)
print(words_sim_bad)

['movie', 'acting', 'good', 'even', 'br']


*-> Commentez ici <-*

Cette fois-ci, si l'on compare nos résultats avec ceux de la pondération PPMI, on remarque que nous obtenons l'inverse. La distance euclidéenne donne de meilleurs résultats que la mesure cosine. 
Pour la distance euclidéenne, on voit clairement que le champ lexical du cinéma est présent avec les mots "episode", "movie", "show" et "series". Le mot "br" provient du fait que nous retrouvons souvent ce charactère HTML "< br /> < br />". Lors de l'étape du prétraitement des données, les caractères de ponctuation seront retirés et, par conséquent, on se retrouve avec br.
Pour la distance cosine, les mots sont plus générals. En effet, on voit movie et acting qui sont dans le champ lexical du cinéma. Ensuite, le mots good est un antonyme de bad. Le mot even est à connotation neutre. 

## 4. Classification de documents avec un modèle de langue

En vous inspirant de [cet article](https://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139), entraînez deux modèles de langue $n$-gramme de caractère avec lissage de Laplace, l'un sur le corpus `pos`, l'autre sur le corpus `neg`. Puis, pour chaque document $D$, calculez sa probabilité selon vos deux modèles : $P(D \mid \textrm{pos})$ et $P(D \mid \textrm{neg})$.

Vous pourrez alors prédire sa classe $\hat{c}_D \in (\textrm{pos}, \textrm{neg})$ en prenant :

$$\hat{c}_D = \begin{cases}
\textrm{pos} & \textrm{si } P(D \mid \textrm{pos}) > P(D \mid \textrm{neg}) \\
\textrm{neg} & \textrm{sinon}
\end{cases}$$

In [37]:
from collections import *

In [38]:
def train_char_lm(data_train, order=4):
    lm = defaultdict(Counter)
    dictionnary = defaultdict(lambda: 0)
    
    pad = "~" * (order - 1)
    for review in data_train:
        review = pad + review
        for i in range(len(review)-(order - 1)):
            history, char = review[i:i+(order - 1)], review[i+(order - 1)]
            dictionnary[char] += 1
            lm[history][char] += 1

    dictionnary['unk'] = 0
    lm['unk']['unk'] = 1 / len(dictionnary)
    
    def normalize(counter):
        s = float(sum(counter.values()))
        return {c:(cnt + 1) / (s + len(dictionnary)) for c,cnt in counter.items()}
            
    
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm

In [39]:
lm_pos = train_char_lm(data[2], order=4)

In [40]:
lm_neg = train_char_lm(data[3], order=4)

In [41]:
def get_proba(lm, review, order=4):
    proba = 1.0
    pad = "~" * (order - 1)
    review = pad + review
    for i in range(len(review)-(order - 1)):
        history, char = review[i:i+(order - 1)], review[i+(order - 1)]
        try:
            proba *= (-1/math.log(lm[history][char]))
        except:
            proba *= (-1/math.log(lm['unk']['unk']))
        
    return proba

## 5. Classification de documents avec sac de mots et Naive Bayes

Ici, vous utiliserez l'algorithme Multinomial Naive Bayes (disponible dans [`sklearn.naive_bayes.MultinomialNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)) pour classifier les documents. Vous utiliserez un modèle sac de mots (en anglais *bag of words*, ou BoW) avec TF-IDF pour représenter vos documents.

*Note :* vous avez déjà construit la matrice TF-IDF à la section 2.1.

In [42]:
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB

In [43]:
def classifier_BOW_Naive():
    train_data_Y = np.concatenate((np.ones(len(data[2])), np.zeros(len(data[3]))))
    test_data_X = np.asarray(data[0] + data[1])
    test_data_Y = np.concatenate((np.ones(len(data[0])), np.zeros(len(data[1]))))
    
    clf = MultinomialNB(alpha=0.5)
    clf.fit(matrix_TFIDF, train_data_Y)    
    matrix_TFIDF_test = create_matrix_TFIDF(matrix_BOW,test_data_X)
    y_pred = clf.predict(matrix_TFIDF_test)
    score = f1_score(test_data_Y, y_pred)
                                                  
    return score, y_pred, test_data_Y

In [44]:
score_BOW_Naive = classifier_BOW_Naive()[0]

In [45]:
print(score_BOW_Naive)

0.869164126423873


## 6. Améliorations

Ici, vous devez proposer une méthode d'amélioration pour le modèle précédent, la justifier et l'implémenter.

*-> Écrivez vos explications ici <-*

Un bon moyen serait de rajouter une étape à notre pré-traitement. En effet, si l'on retire tout les mots qui ne sont ni des noms, ni des adjectifs, ni des adverbes et ni des verbes, on garde uniquement les mots qui peuvent décrire une connotation positive ou négative. Cela a aussi pour effet de retirer plusieurs mots du dictionnaire.

In [46]:
def remove_non_noun_adj():
    new_unigrams_top_5000_words = []
    
    word_type = nltk.pos_tag(unigrams_top_5000_words, tagset='universal')
    for word in word_type:
        if word[1] == 'NOUN' or word[1] == 'ADJ' or word[1] == 'ADV' or word[1] == 'VERB':
            new_unigrams_top_5000_words.append(word[0])
    
    return new_unigrams_top_5000_words

In [47]:
new_unigrams_top_5000_words = remove_non_noun_adj()

In [48]:
matrix_BOW_amelioration = count_BOW(new_unigrams_top_5000_words, data_train_pos_neg)

In [49]:
def classifier_BOW_Naive_Amelioration():
    # train_data_X = data[2] + data[3]
    train_data_Y = np.concatenate((np.ones(len(data[2])), np.zeros(len(data[3]))))
    test_data_X = np.asarray(data[0] + data[1])
    test_data_Y = np.concatenate((np.ones(len(data[0])), np.zeros(len(data[1]))))
    
    matrix_TFIDF_amelioration = create_matrix_TFIDF(matrix_BOW_amelioration, data_train_pos_neg)
    clf = MultinomialNB(alpha=0.5)
    clf.fit(matrix_TFIDF_amelioration, train_data_Y)    
    matrix_TFIDF_test = create_matrix_TFIDF(matrix_BOW_amelioration, test_data_X)
    y_pred = clf.predict(matrix_TFIDF_test)
    score = f1_score(test_data_Y, y_pred)
                                                  
    return score, y_pred, test_data_Y

In [50]:
score_BOW_Naive_Amelioration = classifier_BOW_Naive_Amelioration()[0]

In [51]:
print(score_BOW_Naive_Amelioration)

0.8695582458109518


## 7. Évaluation

Évaluation des modèles des sections 4, 5, 6 sur les données de test. On attend les métriques suivantes : *accuracy*, et pour chaque classe précision, rappel, score F1. Vous pourrez utiliser le module [`sklearn.metrics`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

In [52]:
def get_all_reviews_prob():
    test_data_X = np.asarray(data[0] + data[1])
    test_data_Y = np.concatenate((np.ones(len(data[0])), np.zeros(len(data[1]))))
    
    probas = []
    for review in test_data_X:
        proba_pos = get_proba(lm_pos, review)
        proba_neg = get_proba(lm_neg, review)
        if (proba_pos > proba_neg):
            probas.append(1)
        else:
            probas.append(0)
    
    return probas, test_data_Y
    

In [53]:
from sklearn.metrics import classification_report

In [54]:
y_pred_sec4, y_true_sec4 = get_all_reviews_prob()
print('Section 4')
print(classification_report(y_true_sec4, y_pred_sec4))

Section 4
              precision    recall  f1-score   support

         0.0       0.75      0.88      0.81     12500
         1.0       0.86      0.71      0.77     12500

    accuracy                           0.79     25000
   macro avg       0.80      0.79      0.79     25000
weighted avg       0.80      0.79      0.79     25000



In [55]:
score_sec5, y_pred_sec5, y_true_sec5 = classifier_BOW_Naive()
print('Section 5')
print(classification_report(y_true_sec5, y_pred_sec5))

Section 5
              precision    recall  f1-score   support

         0.0       0.87      0.87      0.87     12500
         1.0       0.87      0.87      0.87     12500

    accuracy                           0.87     25000
   macro avg       0.87      0.87      0.87     25000
weighted avg       0.87      0.87      0.87     25000



In [56]:
score_sec6, y_pred_sec6, y_true_sec6 = classifier_BOW_Naive_Amelioration()
print('Section 6')
print(classification_report(y_true_sec6, y_pred_sec6))

Section 6
              precision    recall  f1-score   support

         0.0       0.87      0.87      0.87     12500
         1.0       0.87      0.87      0.87     12500

    accuracy                           0.87     25000
   macro avg       0.87      0.87      0.87     25000
weighted avg       0.87      0.87      0.87     25000



Commentez vos résultats.

*-> Commentez ici vos résultats <-*
La valeur de 0.0 correspond à "négatif" et la valeur de 1.0 correspond à "positif"

Nous avons testé nos résultats dans les trois cas. Dans le premier cas, nous obtenons 12580 valeurs positives et 12420 valeurs négatives. Si nous affichons les valeurs du vecteur y_pred_sec4, on peut voir que les valeurs sont réparties de manière quasiment aléatoire. Cela explique la raison pour laquelle nous obtenons 0.50.

Pour les sections 5 et 6, on peut voir que nos valeurs sont beaucoup plus précises. En effet, nous avons affiché les valeurs du vecteur y_pred_sec5 et on peut voir que nos 12500 premières valeurs (les reviews positives) contiennent 10835 valeurs positives (environ 87% de succés). Dans les 12500 autres valeurs, on obtient 10903 valeurs négatives (environ 87% de succés). 

